# Calculate yield statistics on simulation datasets

Statistical analysis consists of two stages of the topological gap protocol (TGP) described in Refs. [arXiv:2103.12217](https://arxiv.org/abs/2103.12217) and [arXiv:2207.02472](https://arxiv.org/abs/2207.02472):
* TGP Stage 1 identifies promising regions with stable zero bias peaks (ZBPs) and transfers these regions to TGP Stage 2;
* TGP Stage 2 searches clusters with stable ZBPs, topological gap, and surrounded by gapless boundary.

The simulation is performed for 0 mK, therefore we first temperature broaden the data. This temperature broadening is CPU intensive and will take ≈1 min per dataset. We save the broadened datasets in `simulated/yield/cached` such that it only has to be done once.

Note that we use 30 mK for broadening in Stage 1 and 40 mK in Stage 2. This is due to historical reasons. Increasing Stage 1 temperature to 40 mK barely changes regions for Stage 2.

In [ ]:
from functools import partial
from pathlib import Path
from yield_analysis import analyze_1, analyze_2, parallel_map, show_roi2_tables, show_soi2_tables

import pandas as pd

In [ ]:
folder = Path("../data/simulated/yield/")
folder_stage1 = folder / "stage1"
folder_stage2 = folder / "stage2"

fnames_stage1 = sorted(folder_stage1.glob("*.nc"))
print(f"{len(fnames_stage1)} .nc files total for stage 1")

fnames_stage2 = sorted(folder_stage2.glob("*.nc"))
print(f"{len(fnames_stage2)} .nc files total for stage 2")

if len(fnames_stage1) == 0 or fnames_stage1[0].stat().st_size < 1e6:
    raise Exception(f"You are missing the data, please see the Git LFS instructions at `{str(folder)}/README.md`")

# TGP Stage 1
Now we will perform the analysis on all of these datasets.
*(Stage 1 can take 3-10 minutes on a recent laptop.)*

In [ ]:
T_mK = 30
f = partial(analyze_1, T_mK=T_mK)
results_1 = parallel_map(f, fnames_stage1)
df_1 = pd.DataFrame(results_1)
df_1_passed = df_1[~df_1.V_min.isna()]

In [ ]:
# Print all the bounding boxes
cols = [
    "sample_name",
    "disorder_seed",
    "geometry_seed",
    "surface_charge",
    "B_min",
    "V_min",
    "B_max",
    "V_max",
]
df_1_passed[cols]

In [ ]:
# Print the number of passed devices
passed_devices = df_1_passed.groupby(["sample_name", "surface_charge"]).apply(len)
total = df_1.groupby(["sample_name", "surface_charge"]).apply(len)
df = pd.concat([passed_devices, total], axis=1)
df.columns = ["# passed", "total"]
df

# TGP Stage 2
*(Stage 2 can take 2-4 hours on a recent laptop.)*

Change `B_max` to `2.5` to get the statistics for the "Yield <2.5T" column.

In [ ]:
T_mK = 40
f = partial(analyze_2, T_mK=T_mK, B_max=3.0, return_datasets=True)
all_results = parallel_map(f, fnames_stage2)
results = [r for r in all_results if r is not None]

In [ ]:
df_stats = pd.DataFrame(results)
show_roi2_tables(df_stats)

In [ ]:
show_soi2_tables(df_stats)